## Embedding a Bokeh server in a Notebook

This notebook shows how a Bokeh server application can be embedded inside a Jupyter notebook. 

In [ ]:
import numpy as np

from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Slider
from bokeh.plotting import figure
from bokeh.io import show

There are various application handlers that can be used to build up Bokeh documents. For example, there is a `ScriptHandler` that uses the code from a `.py` file to produce Bokeh documents. This is the handler that is used when we run `bokeh serve app.py`. Here we are going to use the lesser-known `FunctionHandler`, that gets configured with a plain Python function to build up a document. 

Here is the function `modify_doc(doc)` that defines our app:

In [ ]:
def modify_doc(doc):
    x = np.linspace(0.0001, 10, 1000)
    y = np.log(x) * np.sin(x)

    source = ColumnDataSource(data=dict(x=x, y=y))

    plot = figure()
    plot.line('x', 'y', source=source)

    slider = Slider(start=1, end=10, value=1, step=0.1)

    def callback(attr, old, new):
        y = np.log(x) * np.sin(x*slider.value)
        source.data = dict(x=x, y=y)
    slider.on_change('value', callback)
    
    doc.add_root(column(slider, plot))

We take the function above and configure a `FunctionHandler` with it. Then we create an `Application` that uses handler. (It is possible, but uncommon, for Bokeh applications to have more than one handler.) The end result is that the Bokeh server will call `modify_doc` to build new documents for every new sessions that is opened.

In [ ]:
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

handler = FunctionHandler(modify_doc)
app = Application(handler)

Now we can display our application using ``show``:

In [ ]:
show(app)